In [1]:
import datetime
import itertools
import json
import os
import time

import alphashape
import argparse
import cv2
import hdbscan
import igraph as ig

from collections import namedtuple, defaultdict
from matplotlib.path import Path
import matplotlib.pyplot as plt
import metis
import networkx as nx
import numpy as np
import scipy


import pickle
import warnings

from concavehull import concavehull
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import patches
from matplotlib.widgets import CheckButtons, Button, Slider, LassoSelector, RadioButtons, TextBox, RectangleSelector
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from skimage import filters
from sklearn.ensemble import RandomForestClassifier

from hulls import ConcaveHull

import geometry, segmentation as seg, grapher


warnings.simplefilter("ignore")


In [2]:
def points_inside_polygon(polygon, points_arr):
    path = Path(polygon)
    containing = path.contains_points(points_arr)

    return points_arr[containing]

In [3]:
indir = 'unsegmented'
img_id = 0
img = np.asarray(plt.imread(f'{indir}/{str(img_id).zfill(3)}.tif'))
NX, NY = img.shape

In [4]:
img_01 = seg.neighborhood_average(img, d=(5, 5))
img_1 = filters.meijering(img_01)

In [5]:
img_trial = img * (1 - img_1 / np.max(img_1))
img_trial_edges = filters.meijering(img_trial)

### Obtain Clusters

In [6]:
def get_raw_clusters(condition='less_than', threshold=0.2):
    if condition not in ['less_than', 'greater_than']:
        raise ValueError(f'Unsupported condition {condition}')
    if condition == 'less_than':
        features = seg.build_features_matrix(img_trial_edges < threshold, img_trial, 0.05)
    else:
        features = seg.build_features_matrix(img_trial_edges > threshold, img_trial, 0.05)
    clusters_0 = seg.get_clustering_results(features[:, :3], **seg.hdbscan_kwargs)
    clusters = -1 * np.ones(img.shape, dtype=np.int32)
    for i in range(features.shape[0]):
        x, y = [int(v) for v in features[i, :2]]
        clusters[x, y] = clusters_0[i]
    
    return clusters

In [7]:
levels = {
    0: ('less_than', 0.2, lambda size: size >= 0.05),
    1: ('greater_than', 0.05, lambda size: False),
}

points = {}
points_arr = np.zeros((NX * NY, 2), dtype=np.int32)
counter = 0
for ix in range(NX):
    for iy in range(NY):
        idx = int((NX - 1) * iy + ix)
        points[ix, iy] = idx
        points_arr[counter, :] = (ix, iy)
        counter += 1

### Obtain Polygons Bounding Clusters

In [8]:
edges = defaultdict(list)

for level, (condition, threshold, size_check) in levels.items():
    clusters = get_raw_clusters(condition=condition, threshold=threshold)
    for v in np.unique(clusters):
        if v < 0:
            continue
        coords = np.where(np.isclose(clusters, v))
        if coords[0].shape[0] < 5:
                continue
        size = coords[0].shape[0] / 500 ** 2
        if size <= 0:
            continue
        arr = []
        for idx in range(coords[0].shape[0]):
            c = (coords[1][idx], coords[0][idx])
            arr.append(c)
        points_view = {}
        for i in range(len(arr)):
            coord = arr[i]
            idx = int(points[coord])
            points_view[idx] = coord
        graph = grapher.PixelGraph(points=points_view)
        graph.build_graph()
        graph.get_graph_pieces()
        if size_check(size):
            continue
        if size < 0.05:
            hull = []
            try:
                alpha_shape = alphashape.alphashape(arr, 0.2)
                exterior = alpha_shape.exterior
                for c in exterior.coords:
                    hull.append((c[0], c[1]))
            except scipy.spatial._qhull.QhullError as e1:
                pass
            except AttributeError as e2:
                pass
            if len(hull) == 0:
                continue
            hull_ids = []
            for i in range(len(hull)):
                idx = points[hull[i]]
                hull_ids.append(idx)
            edges[level].append(hull_ids)

        else:
            for p in graph.pieces:
                if len(p) < 5:
                    continue
                arr2 = []
                for c in p:
                    arr2.append(points_view[int(c)])
                hull = []
                try:
                    alpha_shape = alphashape.alphashape(arr2, 0.2)
                    exterior = alpha_shape.exterior
                    for c in exterior.coords:
                        hull.append((c[0], c[1]))
                except scipy.spatial._qhull.QhullError as e1:
                    pass
                except AttributeError as e2:
                    pass
                if len(hull) == 0:
                    continue
                hull_ids = []
                for i in range(len(hull)):
                    idx = points[hull[i]]
                    hull_ids.append(idx)
                edges[level].append(hull_ids)

In [9]:
with open(f'segmented/{str(img_id).zfill(3)}.json', 'w') as f:
    json.dump(edges, f)

In [10]:
print(points_inside_polygon(hull, points_arr))

[[169 500]
 [170 500]]
